In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
#import cell
import pandas as pd
import numpy as np
import statistics
import csv
import matplotlib.pyplot as plt
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.bigquery import magics

In [ ]:
BIGQUERY_PROJECT = 'ironhacks-data'
bigquery_client = bigquery.Client(project=BIGQUERY_PROJECT)

In [ ]:
#Gets the master unemployed table
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.unemployment_data`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
unemploymentData = query_job.to_dataframe()
print(unemploymentData.shape)
pd.set_option('display.max_columns', None)
unemploymentData.head(3)

In [ ]:
#Gets each tracks mean and std dev
#unlist has the master unemployment list
#b becomes the filtered list
unList = unemploymentData.values.tolist()
b_set = set(tuple(x) for x in unList)
b = [ list(x) for x in b_set ]

In [ ]:
uuid = []
#makes a list of the unique uuid
for x in b:
    if(uuid.count(x[0]) == 0):
        uuid.append(x[0])

In [ ]:
#setup for extract        
values = []
export = []

In [ ]:
#for each value make a list of each weeks claims
for y in uuid:
    temp = [y]
    for x in b:
        if (x[0] == y):
            temp.append(x[6])
    values.append(temp)

In [ ]:
for x in values:
    name = x[0]
    mean = statistics.mean(x[1:])
    if (len(x) > 2):
        stdev = statistics.stdev(x[1:])
    else:
        print("short")
    export.append([name, mean, stdev])

In [ ]:
#Everything above this is testing
#Make the answer csv
query = """
SELECT *
FROM `ironhacks-data.ironhacks_competition.prediction_list`
"""

In [ ]:
# QUERY THE DATA ONCE
query_job = bigquery_client.query(query)
predictionData = query_job.to_dataframe()

In [ ]:
answers = predictionData.values.tolist()
#Answers[names, week]
#export[names, mean, stddev]
i = 0
for x in answers:
    for y in export:
        if(y.count(x[0]) == 1):
            answers[i].insert(1, y[1])
    i = i + 1

In [ ]:
fields = ['uu_id', 'total_claims', 'week_number']
with open('submission_prediction_output.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(answers)